In [1]:
import cv2
import numpy as np
import pandas as pd

# Hardcode the image path here
img_path = 'img.jpg'

# Reading the image with opencv
img = cv2.imread(img_path)

# Declaring global variables (used later on)
clicked = False
r = g = b = xpos = ypos = 0

# Reading CSV file with pandas and giving names to each column
index = ["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

# Function to calculate minimum distance from all colors and get the most matching color
def getColorName(R, G, B):
    minimum = 10000
    for i in range(len(csv)):
        d = abs(R - int(csv.loc[i, "R"])) + abs(G - int(csv.loc[i, "G"])) + abs(B - int(csv.loc[i, "B"]))
        if d <= minimum:
            minimum = d
            cname = csv.loc[i, "color_name"]
    return cname

# Function to get x, y coordinates of mouse double click
def draw_function(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global b, g, r, xpos, ypos, clicked
        clicked = True
        xpos = x
        ypos = y
        b, g, r = img[y, x]
        b = int(b)
        g = int(g)
        r = int(r)

cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_function)

while True:
    cv2.imshow("image", img)
    if clicked:
        # cv2.rectangle(image, startpoint, endpoint, color, thickness)-1 fills entire rectangle 
        cv2.rectangle(img, (20, 20), (750, 60), (b, g, r), -1)

        # Creating text string to display (Color name and RGB values)
        text = getColorName(r, g, b) + ' R=' + str(r) + ' G=' + str(g) + ' B=' + str(b)

        # cv2.putText(img, text, start, font(0-7), fontScale, color, thickness, lineType)
        cv2.putText(img, text, (50, 50), 2, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        # For very light colors, display text in black color
        if r + g + b >= 600:
            cv2.putText(img, text, (50, 50), 2, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

        clicked = False

    # Break the loop when user hits 'esc' key    
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()


# store color for the detected vehicle


In [3]:
import torch
import cv2
import numpy as np
import mysql.connector
import time
import pandas as pd

# Load color reference CSV and create color lookup
index = ["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

def get_color_name(bgr):
    b, g, r = bgr
    minimum = 10000
    cname = "Unknown"
    for i in range(len(csv)):
        d = abs(r - int(csv.loc[i, "R"])) + abs(g - int(csv.loc[i, "G"])) + abs(b - int(csv.loc[i, "B"]))
        if d <= minimum:
            minimum = d
            cname = csv.loc[i, "color_name"]
    return cname

# Function to store vehicle information in the database
def store_vehicle_info(color, vehicle_type, model, timestamp):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="viz_optilytics"
    )
    cursor = conn.cursor()

    sql = "INSERT INTO vehicles (color, type, model, timestamp) VALUES (%s, %s, %s, %s)"
    values = (color, vehicle_type, model, timestamp)

    cursor.execute(sql, values)
    conn.commit()
    cursor.close()
    conn.close()

# Function to extract color from bounding box
def extract_color(frame, bbox):
    x1, y1, x2, y2 = bbox
    roi = frame[y1:y2, x1:x2]
    average_color = np.mean(roi, axis=(0, 1))
    color_name = get_color_name(average_color)
    return color_name

# Load the YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Capture video from webcam
cap = cv2.VideoCapture(0)

vehicle_data = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    resized_frame = cv2.resize(frame, (640, 480))
    results = model(resized_frame)

    for *box, conf, cls in results.xyxy[0]:
        x1, y1, x2, y2 = map(int, box)
        label = model.names[int(cls)]
        
        vehicle_id = f"{x1}{y1}{x2}{y2}"
        if vehicle_id not in vehicle_data:
            vehicle_data[vehicle_id] = {'start_time': time.time(), 'last_position': (x1, y1, x2, y2)}

        if vehicle_data[vehicle_id]['last_position'] == (x1, y1, x2, y2):
            if time.time() - vehicle_data[vehicle_id]['start_time'] > 2:
                # Extract vehicle details and store them
                vehicle_color = extract_color(frame, (x1, y1, x2, y2))
                vehicle_type = label
                vehicle_model = "Unknown Model"  # Placeholder
                timestamp = time.strftime('%Y-%m-%d %H:%M:%S')

                # Store to database
                store_vehicle_info(vehicle_color, vehicle_type, vehicle_model, timestamp)
                print(f"Vehicle stored: Color={vehicle_color}, Type={vehicle_type}, Model={vehicle_model}, Time={timestamp}")
                
                # Add red alert
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, "Red Alert: Vehicle Stopped", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        else:
            vehicle_data[vehicle_id] = {'start_time': time.time(), 'last_position': (x1, y1, x2, y2)}

    # Render and show the frame
    rendered_frame = np.squeeze(results.render())
    cv2.imshow('YOLOv5 Webcam Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Vehicle stored: Color=Languid Lavender, Type=car, Model=Unknown Model, Time=2024-08-31 14:32:53
Vehicle stored: Color=Languid Lavender, Type=car, Model=Unknown Model, Time=2024-08-31 14:32:53
Vehicle stored: Color=Languid Lavender, Type=car, Model=Unknown Model, Time=2024-08-31 14:32:54
Vehicle stored: Color=Languid Lavender, Type=car, Model=Unknown Model, Time=2024-08-31 14:32:55
Vehicle stored: Color=Languid Lavender, Type=car, Model=Unknown Model, Time=2024-08-31 14:32:55
